# 生成straddle信息, trading day version

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from scipy import interpolate
import math
from scipy.integrate import quad
import Options
from matplotlib import pyplot as plt
import datetime
import time
import tushare

In [17]:
token = '37f756066bdfd9e6973cc51e088dddd1f3576f970fe77da55592be27'
tushare.set_token(token)
pro = tushare.pro_api(token)

trading_days = pro.query('trade_cal',
                         exchange='SSE',
                         start_date='20150101',
                         end_date='20220101')
num_tds = trading_days.iloc[np.where(trading_days['is_open'])].cal_date.values
trading_days = pd.to_datetime(trading_days.iloc[np.where(
    trading_days['is_open'])].cal_date).astype(str).reset_index(
        drop=True).values[::-1]

In [18]:
all_info = pd.read_csv("cleaned_data_50ETF_option.csv", low_memory=False)
shibor_info = pd.read_csv("combined_SHIBOR.csv")
index_info = pd.read_csv('data_50ETF_index.csv')
index_info = index_info[index_info.date >= all_info.date[0]].reset_index(
    drop=True)

all_info['ukey'] = all_info.code.apply(lambda x: int(x.split(".")[0]))

op_info = pd.read_csv('etf_options.csv')
op_info['ukey'] = op_info.ukey.apply(lambda x: int(x[2:]))
op_info = op_info[['DataDate', 'open', 'ukey']]
op_info.columns = ['date', 'open', 'ukey']

all_info = all_info.merge(op_info, on=['ukey', 'date'])

In [19]:
shibor_dates = np.array([1, 5, 10, 21, 63, 126, 189, 252])
info_dict = {'date': [], 'exe_date': [], 'implied_vol': []}
trading_dates = all_info.date.unique()
indexes = index_info.close.values


def abs_diff(x):
    s = x.values[0]
    for k in range(1, len(x)):
        s -= x.values[k]
    return np.abs(s)


implied_index = []
case1, case2 = [], []
all_dict = {day: [] for day in trading_dates}
all_tuples = []
unds = {'date': [], 'mat': [], 'und': []}
ptm_df = pd.DataFrame({'dtm': [], 'call_imp': [], 'put_imp': []})
for td in tqdm(trading_dates):

    td_index = np.where(trading_days == td)[0][0]
    td_shibor = shibor_info[shibor_info.date == td].iloc[0].values[1:]

    shibor_helper = interpolate.interp1d(shibor_dates,
                                         td_shibor,
                                         kind='slinear')

    td_info = all_info[all_info.date == td]

    td_mats = np.sort(td_info.exe_enddate.unique())

    td_shibor = shibor_info[shibor_info.date == td].values[0][1:]

    td_shibor = td_shibor[-1] / 100

    temp_imp = []
    for maturity in td_mats:

        mt_index = np.where(trading_days == maturity)[0][0]

        ptmday = mt_index - td_index + 1

        mat_info = td_info[td_info.exe_enddate == maturity]
        available_prices = mat_info.exe_price.sort_values().unique()
        atm = mat_info.groupby("exe_price").apply(
            lambda x: abs_diff(x['close'])).idxmin()
        call = mat_info[(mat_info.exe_price == atm)
                        & (mat_info.exe_mode == 'call')]
        put = mat_info[(mat_info.exe_price == atm)
                       & (mat_info.exe_mode == 'put')]

        cid, pid = call.code.values[0], put.code.values[0]

        discount = np.exp(-td_shibor * ptmday / 252)
        implied_underlying = call.close.values[0] - put.close.values[
            0] + atm * discount
        temp_imp.append(implied_underlying)
        option = Options.Strategy()
        option.getData(atm, td_shibor, ptmday / 252, 0.2)
        implied_vol = option.EuroCallIV(implied_underlying,
                                        call.close.values[0])
        implied_vol_p = option.EuroPutIV(implied_underlying,
                                         put.close.values[0])

        call_price = call.close.values[0]
        put_price = put.close.values[0]

        call_open = call.open.values[0]
        put_open = put.open.values[0]

        option.setSigma(implied_vol)
        call_delta = option.EuroCallDelta(implied_underlying)

        option.setSigma(implied_vol_p)
        put_delta = option.EuroPutDelta(implied_underlying)
        info_dict = {
            'dtm': [ptmday],
            'call_imp': [implied_vol],
            'put_imp': [implied_vol_p],
            'call_delta': [call_delta],
            'put_delta': [put_delta],
            'date': [td],
            'exe_date': [maturity],
            'call_code': [cid],
            'put_code': [pid]
        }
        info_dict['call'] = call_price
        info_dict['put'] = put_price
        info_dict['call_open'] = call_open
        info_dict['put_open'] = put_open
        tmp = pd.DataFrame(info_dict)
        ptm_df = pd.concat([ptm_df, tmp])
        unds['date'].append(td)
        unds['mat'].append(maturity)
        unds['und'].append(implied_underlying)
    implied_index.append(np.mean(temp_imp))

100%|██████████████████████████████████████████████████████████████████████████████| 1504/1504 [01:10<00:00, 21.41it/s]


In [20]:
ptm_df = ptm_df.merge(pd.DataFrame({
    'date': ptm_df['date'].unique().tolist(),
    'stock_price': implied_index
}), on='date')

In [21]:
options = ptm_df[['date', 'call_code', 'put_code', 'call_imp']]
options = options[(options['date'] < '2021-04-14')]
options.columns = ['date', 'call_code', 'put_code', 'imp']
all_prices = all_info[['date', 'code', 'close', 'open']]
all_prices.columns = ['tmr', 'code', 'next_close', 'next_open']

In [22]:
call_dir = 'new_td//call//'
put_dir = 'new_td//put//'

calls = ptm_df[[
    'date', 'exe_date', 'call_code', 'call_imp', 'call_delta', 'call',
    'call_open', 'stock_price', 'dtm'
]].reset_index(drop=True)
calls.columns = [
    'date', 'exe_date', 'code', 'call_imp', 'call_delta', 'call_close',
    'call_open', 'stock_price', 'dtm'
]
calls['tmr'] = calls.date.apply(
    lambda x: trading_days[np.where(trading_days == x)[0] + 1][0])
calls['weekday'] = pd.to_datetime(calls.date).apply(lambda x: x.weekday())
calls = pd.merge(calls, all_prices, on=['tmr', 'code'])

puts = ptm_df[[
    'date', 'exe_date', 'put_code', 'put_imp', 'put_delta', 'put', 'put_open',
    'stock_price', 'dtm'
]].reset_index(drop=True)
puts.columns = [
    'date', 'exe_date', 'code', 'put_imp', 'put_delta', 'put_close',
    'put_open', 'stock_price', 'dtm'
]
puts['tmr'] = puts.date.apply(
    lambda x: trading_days[np.where(trading_days == x)[0] + 1][0])
puts['weekday'] = pd.to_datetime(puts.date).apply(lambda x: x.weekday())
puts = pd.merge(puts, all_prices, on=['tmr', 'code'])

for k in calls.exe_date.unique():
    selected = calls[calls['exe_date'] == k].reset_index(drop=True)
    selected.to_csv(call_dir + 'info_{}.csv'.format(k), index=False)

for k in puts.exe_date.unique():
    selected = puts[puts['exe_date'] == k].reset_index(drop=True)
    selected.to_csv(put_dir + 'info_{}.csv'.format(k), index=False)

In [23]:
calls = calls[[
    'date', 'exe_date', 'dtm', 'weekday', 'call_imp', 'stock_price', 'code',
    'call_delta', 'call_close', 'call_open', 'next_close', 'next_open'
]]
calls.columns = [
    'date', 'exe_date', 'dtm', 'weekday', 'imp', 'stock_price', 'call_code',
    'call_delta', 'call_close', 'call_open', 'call_next_close',
    'call_next_open'
]

puts = puts[[
    'date', 'exe_date', 'dtm', 'weekday', 'put_imp', 'stock_price', 'code',
    'put_delta', 'put_close', 'put_open', 'next_close', 'next_open'
]]
puts.columns = [
    'date', 'exe_date', 'dtm', 'weekday', 'imp', 'stock_price', 'put_code',
    'put_delta', 'put_close', 'put_open', 'put_next_close', 'put_next_open'
]

straddles = pd.merge(
    calls, puts, on=['date', 'exe_date', 'dtm', 'weekday', 'stock_price', 'imp'])

In [24]:
straddles.to_csv('all_straddles_td.csv', index=False)